In [9]:
import cobra
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path
import pickle


In [10]:
import pytfa

from optlang.exceptions import SolverError

from cobra.flux_analysis import flux_variability_analysis
from cobra.io import load_matlab_model, load_json_model


from pytfa.io import import_matlab_model, load_thermoDB,                    \
                            read_lexicon, annotate_from_lexicon,            \
                            read_compartment_data, apply_compartment_data

# Build iJO1366 TFA model

In [11]:
solver = 'optlang_gurobi'
pytfa_folder = Path("C:/Users/snorres/git/pytfa")
thermo_data = load_thermoDB(pytfa_folder / "data" / "thermo_data.thermodb")

In [12]:
 # We import pre-compiled data as it is faster for bigger models
cobra_model = load_json_model(pytfa_folder / 'models/iJO1366.json')
# The lexicon is just a csv file with seed annotat
lexicon = read_lexicon(pytfa_folder / 'models/iJO1366/lexicon.csv')


In [13]:
compartment_data = read_compartment_data(str(pytfa_folder / 'models/iJO1366/compartment_data.json'))



In [14]:
# Initialize the cobra_model
mytfa = pytfa.ThermoModel(thermo_data, cobra_model)

# Annotate the cobra_model
annotate_from_lexicon(mytfa, lexicon)
apply_compartment_data(mytfa, compartment_data)

biomass_rxn = 'Ec_biomass_iJO1366_WT_53p95M'

Read LP format model from file C:\Users\snorres\AppData\Local\Temp\tmpsczurt1f.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20334 nonzeros


2023-02-01 12:37:14,605 - thermomodel_None - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [15]:
mytfa.name = 'leakage'
mytfa.solver = solver
mytfa.objective = biomass_rxn
mytfa.description = "Model used for predicting shadow prices"

In [16]:
def apply_solver_settings(model, solver = solver):
    model.solver = solver
    # model.solver.configuration.verbosity = 1
    model.solver.configuration.tolerances.feasibility = 1e-9
    if solver == 'optlang_gurobi':
        model.solver.problem.Params.NumericFocus = 3
    model.solver.configuration.presolve = True

apply_solver_settings(mytfa)

In [17]:
## FBA
fba_solution = cobra_model.optimize()
fba_value = fba_solution.objective_value
# fva = flux_variability_analysis(mytfa)

In [18]:
## TFA conversion
mytfa.prepare()
mytfa.convert()#add_displacement = True)

2023-02-01 12:37:14,837 - thermomodel_None - INFO - # Model preparation starting...
2023-02-01 12:37:17,992 - thermomodel_None - WARNING - Warning : NULL/U
2023-02-01 12:37:17,992 - thermomodel_None - WARNING - Warning : NULL/L
2023-02-01 12:37:17,992 - thermomodel_None - WARNING - Warning : NULL/L
2023-02-01 12:37:17,992 - thermomodel_None - WARNING - Warning : NULL/U
2023-02-01 12:37:17,992 - thermomodel_None - WARNING - Warning : NULL/L
2023-02-01 12:37:17,992 - thermomodel_None - WARNING - Warning : NULL/L
2023-02-01 12:37:18,270 - thermomodel_None - WARNING - Warning : NULL/U
2023-02-01 12:37:18,271 - thermomodel_None - WARNING - Warning : NULL/L
2023-02-01 12:37:18,271 - thermomodel_None - WARNING - Warning : NULL/L
2023-02-01 12:37:18,273 - thermomodel_None - WARNING - Warning : NULL/U
2023-02-01 12:37:18,274 - thermomodel_None - WARNING - Warning : NULL/L
2023-02-01 12:37:18,276 - thermomodel_None - WARNING - Warning : NULL/L
2023-02-01 12:37:18,491 - thermomodel_None - WARNING

In [19]:
## Info on the cobra_model
mytfa.print_info()


                                                   value
key                                                     
name                                             leakage
description      Model used for predicting shadow prices
num constraints                                    15034
num variables                                      15538
num metabolites                                     1807
num reactions                                       2585
                               value
key                                 
num metabolites(thermo)  1550.000000
num reactions(thermo)    1824.000000
pct metabolites(thermo)    85.777532
pct reactions(thermo)      70.560928


In [20]:
## Optimality
tfa_solution = mytfa.optimize()
tfa_value = tfa_solution.objective_value

In [21]:
if tfa_value < 0.1:
    from pytfa.optim.relaxation import relax_dgo

    mytfa.reactions.get_by_id(biomass_rxn).lower_bound = 0.5*fba_value
    relaxed_model, slack_model, relax_table = relax_dgo(mytfa)

    original_model, mytfa = mytfa, relaxed_model

    print('Relaxation: ')
    print(relax_table)
    
    tfa_solution = mytfa.optimize()
    tfa_value = tfa_solution.objective_value

Read LP format model from file C:\Users\snorres\AppData\Local\Temp\tmp2qyo3g3u.lp
Reading time = 0.03 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2023-02-01 12:44:07,330 - thermomodel_leakage - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Read LP format model from file C:\Users\snorres\AppData\Local\Temp\tmpv8cyu43l.lp
Reading time = 0.05 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


2023-02-01 12:51:18,919 - thermomodel_leakage - INFO - # Model initialized with units kcal/mol and temperature 298.15 K
2023-02-01 12:51:40,639 - thermomodel_leakage - INFO - Adding slack constraints
adding slacks: 100%|███████████████████████████████████████████████████████████████| 1824/1824 [00:41<00:00, 44.29it/s]
2023-02-01 12:52:26,473 - thermomodel_leakage - INFO - Optimizing slack model
2023-02-01 12:52:31,660 - thermomodel_leakage - INFO - Extracting relaxation
applying slack: 100%|██████████████████████████████████████████████████████████| 2585/2585 [00:00<00:00, 113636.40it/s]
2023-02-01 12:52:31,983 - thermomodel_leakage - INFO - Testing relaxation


Relaxation: 
              lb_in      ub_in  lb_change  ub_change     lb_out     ub_out
AIRC3    -11.941085  -8.988685   0.000000   2.582409 -11.941085  -6.406276
ARGSL     16.770653  18.636729   2.184361   0.000000  14.586293  18.636729
ATPS4rpp  -9.518201  -8.083092   1.517942   0.000000 -11.036143  -8.083092
GALT1     67.754735  69.018800  54.942183   0.000000  12.812552  69.018800
HEPK1      6.478044   6.951504   0.071768   0.000000   6.406276   6.951504
HEPK2     44.872230  45.345690  32.059679   0.000000  12.812552  45.345690
HEPT1     25.810459  27.074524  12.997907   0.000000  12.812552  27.074524
HEPT2      7.030447   8.294512   0.624171   0.000000   6.406276   8.294512
MECDPS    26.135683  30.646876  11.549390   0.000000  14.586293  30.646876
MTHTHFSs  13.297862  14.814659   6.891586   0.000000   6.406276  14.814659


In [22]:
tfa_value

0.8031193596364457

In [23]:
tfa_solution = mytfa.optimize()

# Predict shadow prices

In [ ]:
def predict_shadow_price

# Load 

# Store model

In [ ]:
# Can't write model
# cobra.io.write_sbml_model(mytfa,'models/e_coli/iJO1366_tfa.xml')
#with open("models/e_coli/iJO1366_tfa.obj", "wb") as f:
#    pickle.dump(mytfa, f)

In [ ]:
cobra.io.save_yaml_model(mytfa, "models/e_coli/iJO1366_tfa.yml")

In [ ]:
#¤import json
#with open("models/e_coli/iJO1366_tfa.json", "w") as f:
#    json.dump(mytfa, f)

# Load model

In [8]:
with open("models/e_coli/iJO1366_tfa.obj", "rb") as f:
    mytfa2 = pickle.load(f)

Read LP format model from file C:\Users\snorres\AppData\Local\Temp\tmpqov5zeid.lp
Reading time = 0.07 seconds
: 15034 rows, 15538 columns, 52691 nonzeros
Read LP format model from file C:\Users\snorres\AppData\Local\Temp\tmpw5p14h75.lp
Reading time = 0.02 seconds
: 1807 rows, 5170 columns, 20304 nonzeros


TypeError: __new__() takes 2 positional arguments but 6 were given

In [51]:
for m_id in ["g6p_c", "pyr_c", "ac_c"]:
    with mytfa as model:
        m = model.metabolites.get_by_id(m_id)
        r = model.add_boundary(m, type = "demand")
        r.bounds = (0.01, 0.01)
        solution = model.slim_optimize()
        print(solution, tfa_value-solution)

0.8020941357807011 0.0010252238557445503
0.802729774571263 0.00038958506518260716
0.8028425491953959 0.0002768104410497685


(0, 1000.0)